In [1]:
import torch
#from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from Optim import Optim
import torch.optim as optim
import argparse
import random
import numpy as np
import timeit
from models import GCN, ChebyNet, MoNet, DSGC
from utils import computeLaplacian
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from sklearn.preprocessing import normalize

from CustomCVs import StratifiedKFoldMixedSizes, StratifiedKFoldByGroups
from data_handling_Sev import create_data_set
from sklearn.model_selection import GridSearchCV, GroupKFold, LeaveOneGroupOut
from scipy.spatial.distance import cdist

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score,cross_val_predict,StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

# parser 객체 생성
parser = argparse.ArgumentParser(description='PyTorch Time series forecasting')
# argument 추가
parser.add_argument('--model', type=str, default='DPIEnn' ,help='Model Name')
parser.add_argument('--epochs', type=int, default=60,help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=128, metavar='N',help='batch size')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout applied to layers (0 = no dropout)')
parser.add_argument('--seed', type=int, default=1234,help='random seed')
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--save', type=str,  default='save/model.pt',help='path to save the final model')
parser.add_argument('--cuda', type=str, default=True, help='use gpu or not')
parser.add_argument('--nn', type=int, default=16, help='number of the nearest neighbors')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--sigma', type=float, default=None, help='sigma coefficient.')
parser.add_argument('--embedding', type=int, default=40, help='embedding_num')
parser.add_argument('--cv', type=int, default=0, help='cv type: 0, 1, 2')
parser.add_argument('--numlayers', type=int, default=2, help='# of layers')

#parser.add_argument('--nn', type=str, default='8,8,8,8,3', help='number of the nearest neighbors')
#parser.add_argument('--l', type=float, default=0.25, help='laplacian constant for testing')

# method로 명령창에서 주어진 인자를 args 이름으로 파싱
#jupyter notebook에서는 이 부분 에러 발생
# : argparse 라이브러리를 사용하기 원한다면 터미널이나 다른 프레임워크에서 실행
#args = parser.parse_args()

import easydict
 
args = easydict.EasyDict({
    "model": "GCN",
    "epochs": 60,
    "batch_size": 128,
    "dropout": 0.5,
    "seed": "1234",
    "gpu": 0,
    "save": 'save/model.pt',
    "cuda": True,
    "nn": 16,
    "lr": 0.001,
    "sigma": None,
    "embedding": 40,
    "cv": 0,
    "numlayers": 2,
})

print('args', args)

# Data
print('==> Preparing data..')

args {'epochs': 60, 'cv': 0, 'nn': 16, 'model': 'GCN', 'numlayers': 2, 'gpu': 0, 'batch_size': 128, 'save': 'save/model.pt', 'embedding': 40, 'dropout': 0.5, 'sigma': None, 'lr': 0.001, 'cuda': True, 'seed': '1234'}
==> Preparing data..


In [2]:
#set a device
# gpu/cpu를 통해서 학습 및 test 실행
device = torch.device('cuda:'+str(args.gpu) if torch.cuda.is_available() else 'cpu')
print(device)

cudnn.benchmark = True
bShowParameter = False

if bShowParameter:
    for name, param in model.named_parameters():
        if param.requires_grad:      # requires_grad == True : tensor의 모든 연산에 대하여 추적& 기록
            print(name, param.data.size())

# #print # of parameters
# nParams = sum([p.nelement() for p in model.parameters()])
# print('* number of parameters: %d' % nParams)

def miniBatchDat(X,y,batchSize = 128,del_idx=None):
    n = len(X)
    # X = normalize(X, axis=0, norm='l1')

    # n개의 data를 batchSize별로 나누고 그 batch들의 시작점마다
    for begin in range(0, n, batchSize):
        # y = torch.Tensor(y[begin:begin + batchSize]).type(torch.LongTensor)
        if not del_idx:
            # X, y data에서 나눠진 batch들의 Tensor를 2차원으로 
            yield torch.unsqueeze(torch.Tensor(X[begin:begin + batchSize, :]),2), torch.Tensor(y[begin:begin + batchSize]).long()
        else:
            X1 = np.delete(X[begin:begin + batchSize, :],del_idx,1)
            X2 = X[begin:begin + batchSize, del_idx]
            yield torch.unsqueeze(torch.Tensor(X1),2), torch.Tensor(X2), torch.Tensor(y[begin:begin + batchSize]).long()

# 가중치 초기화
def weights_init(m):
    if type(m) == nn.Linear:
        m.reset_parameters()

#training
def train(model,epoch,X,y):
    # training 시작 시간
    time_st = timeit.default_timer()
    # 학습
    model.train()
    
    train_loss = 0
    correct = 0
    total = 0
    batch_idx = 0
    
    # batch로 나눠서 batch 순서대로 학습 진행 + loss 계산 + optimize
    for in1,in2, targets in miniBatchDat(X,y,args.batch_size,del_idx):
        in1,in2, targets = in1.to(device),in2.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(in1,in2)
        
        #print(outputs, targets)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        batch_idx += 1

    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print(name, param.data, param.grad)
    train_time = timeit.default_timer() - time_st
    print('[%3d/%4d] '%(epoch, args.epochs),'Training Time: %2f'% train_time, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'%(train_loss/(batch_idx+1),100.*correct/total, correct, total), end =" ")


#testing
def test(model, X,y):
    time_st = timeit.default_timer()
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        batch_idx = 0
        for in1, in2, targets in miniBatchDat(X, y, args.batch_size, del_idx):
            in1, in2, targets = in1.to(device), in2.to(device), targets.to(device)
            outputs = model(in1,in2)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_idx += 1
    test_time = timeit.default_timer() - time_st
    test_acc = 100.*correct/total

    print('Testing Time: %2f'% test_time, 'Acc: %.3f%% (%d/%d)'%(test_acc, correct, total))
    return test_acc

best_acc = 0.0
# best_epoch = 0
cv_type = 0

cuda:0


In [3]:
# Here we will import our data from the csv: we do NOT drop out participants with incomplete features (complete=False), but they should have at least 90% feature completeness (completeness_threshold=0.9), we add Age and Sex and SiteID (age_group_tesla_site) as extra columns (covariates), and we automaticly exclude sites with too little participants (min_counts_per_site). I added documentation to this function, check it for further details.

# ENIGMA_OCD_26-01-2019.csv 파일 upload하여 data 이용
X,fs_labels,cov,y, groups = create_data_set(complete=True, completeness_threshold=0.9,
                        covariates=['Age', 'Sex', 'age_group_tesla_site'], min_counts_per_site='auto')
#print('x:{},fs_labels:{},cov:{},y:{},groups:{}'.format(X,fs_labels,cov,y,groups))
# X : 각 환자별로 정보 저장
# y : 강박증 여부 (0-강박증x / 1-강박증ㅇ)
# fs_labels : 뇌 구조와 관련된 열의 이름 값
# groups : age_group_tesla_site 열의 모든 행 값

print('X shape:{}'.format(X.shape))
#print(relevant_features)
#print(cov)

# fs_labels : csv에서 사용된 feature들 저장된 배열
#print(fs_labels)

feat_surf = []
feat_thick = []

feat_idx = []
x_del_idx = []

del_idx = []

# for i, feat in enumerate(fs_labels):
#     feature = feat.split('_')
#     # print(feature)
#     feat_name = feature[0]
#     feat_type = None
#     if len(feature) == 3:
#         feat_type = feature[2]
#     # print(len())
#     if feat_type:
#         # print('no')
#         if feat_type == 'surfavg':
#             feat_surf.append(feat)
#         elif feat_type == 'thickavg':
#             del_idx.append(i)
#             feat_thick.append(feat)
#     else:
#         feat_surf.append(feat)
#         feat_thick.append(feat)

label_group = {}

#group labels: 1 - subcortical, 2 - cortical w/ surface, 3 - cortical w/ thickness
for i, feat in enumerate(fs_labels):
    feature = feat.split('_')  # L/R + 뇌 구조 형식으로 분리
    feat_name = feature[0]     # L/R
    feat_type = None
    if len(feature) == 3:
        feat_type = feature[2]   # feat_type != True -> group = 1
    if feat_type:
        if feat_type == 'surfavg':
            label_group[feat] = 2
        elif feat_type == 'thickavg':
            label_group[feat] = 3
    else:
        label_group[feat] = 1
        
# label_group : fs_labels의 feature들이 속한 label 각각 저장되어 있는 배열
#print(label_group)

def computeDist(df,feat1,feat2,min,max,group):
    # dtype='f8' : a, b 배열을 float64로 맞춰주기 위한 코드
    a = np.array(df.loc[feat1].tolist()[:-1], dtype='f8')  # .reshape(1, -1)
    b = np.array(df.loc[feat2].tolist()[:-1], dtype='f8')  # .reshape(1, -1)
    
    for idx in range(3):
        a[idx] = (a[idx] - min[group][idx])/(max[group][idx] - min[group][idx])
        b[idx] = (b[idx] - min[group][idx])/(max[group][idx] - min[group][idx])

    return np.linalg.norm(a - b)

# 각 label별 min, max 계산 + 저장
def computeMinMax(row,min,max,group):
    for idx in range(3):
        if min[group][idx] > float(row[idx]):
            min[group][idx] = float(row[idx])
        if max[group][idx] < float(row[idx]):
            max[group][idx] = float(row[idx])
    return min,max

Loading ENIGMA dataset: Complete = True, Min_Threshold = 0.9, Age_Group = None, Covariates = ['Age', 'Sex', 'age_group_tesla_site'], Min_Counts_per_Site = auto, y_label = None 

Finished loading data set: 225 samples, 159 FS features, 3 covariates 
 
 
X shape:(225, 159)


/home/rlatn/.local/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/rlatn/team1/data_handling_Sev.py:199: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X = tmp_df.loc[:, fs_labels].as_matrix()


In [4]:
if args.model == 'GCN' or args.model == 'DSGC' or args.model == 'ChebyNet':

    df = pd.read_csv('./dat/dictionary.csv', index_col=0)
    #print(df.index)

    dist = np.zeros((fs_labels.shape[0], fs_labels.shape[0]))
    max = [[-9999,-9999,-9999],[-9999,-9999,-9999],[-9999,-9999,-9999]]
    min = [[9999,9999,9999],[9999,9999,9999],[9999,9999,9999]]

    del_idx = []
    index = df.index
    #print(df.index.values)

    for feat1 in index:
        # if label_group[feat1] == 1:
        row = df.loc[feat1].tolist()[:-1]
        if(not (df.loc[feat1].tolist()[3]=='skip' or df.loc[feat1].tolist()[3]==' skip')):
            #print(row)
            min,max = computeMinMax(row,min,max,label_group[feat1] - 1)


    #remove LSurfArea,RSurfArea,LThickness,RThickness
    for i, feat1 in enumerate(fs_labels):
        
        if feat1 in index:
            for j, feat2 in enumerate(fs_labels):
                if feat2 in index:
                    
                    if label_group[feat1] == 1:
                        if label_group[feat2] == 1:
                            dist[i, j] = computeDist(df,feat1,feat2,min,max,0)
                        elif label_group[feat2] == 2 or label_group[feat2] == 3:
                            dist[i, j] = 1.0
                            
                    elif label_group[feat1] == 2:
                        if label_group[feat2] == 1:
                            dist[i, j] = 1.0
                        elif label_group[feat2] == 2:
                            dist[i, j] = computeDist(df, feat1, feat2,min,max,1)
                            
                    elif label_group[feat1] == 3:
                        if label_group[feat2] == 1:
                            dist[i, j] = 1.0
                        elif label_group[feat2] == 3:
                            dist[i, j] = computeDist(df, feat1, feat2,min,max,2)
        else:
            del_idx.append(i)
            #print(feat1): subcort_ICV & cort_ICV : ENIGMA data에는 존재 but dictionary에는 존재x feature
            
    dist = np.delete(dist,del_idx,axis = 0)
    dist = np.delete(dist,del_idx,axis = 1)

    print(dist.shape)
    print(dist)
    print(del_idx)
    #print(np.sum(np.isnan(dist)))
    # X = np.delete(X,del_idx,1)

    # 각 feature별로 가장 가까운 순서대로 feature의 index 저장
    print(np.argsort(dist)[:, 1:args.nn+1] )

(156, 156)
[[0.         0.95245041 0.46841652 ... 1.         1.         1.        ]
 [0.95245041 0.         0.67339195 ... 1.         1.         1.        ]
 [0.46841652 0.67339195 0.         ... 1.         1.         1.        ]
 ...
 [1.         1.         1.         ... 0.         0.80107009 0.2792698 ]
 [1.         1.         1.         ... 0.80107009 0.         0.52712377]
 [1.         1.         1.         ... 0.2792698  0.52712377 0.        ]]
[0, 1, 158]
[[  4   2   5 ... 110 115 109]
 [  9   6   3 ... 105 106 107]
 [  0   4   5 ... 113 112 111]
 ...
 [ 72  50  49 ...  38  44  37]
 [ 63  62  61 ...  49  48  47]
 [ 62  61  60 ...  48  47  46]]


In [5]:
"""
유전자 data decode/encode -> team3
ocd에서는 영향 x
abcd, conn data와 같이 큰 data non-lin
        
<data pre-processing>
age에 큰 영향, age: non-linear
1. subcort + cor_ICV = total_ICV -> 분모 / 분자 : feature들 -> ratio 이용
   -> age related된 effect 보정 가능 예상
      thickness 는 잘 보정 x
      missing data 많은 col 찾기
   => 보정하고 다시 학습 진행
2. 10대가 많으면 , 나눠서 실험
3. 23개 feature -> dictionary.csv 파일 feature들로 새로 생성
   개수 적으면 RF에서 강력 -> light model이 더 잘나올듯
   gcn & 다른 모델 학습 진행
"""

'\n유전자 data decode/encode -> team3\nocd에서는 영향 x\nabcd, conn data와 같이 큰 data non-lin\n        \n<data pre-processing>\nage에 큰 영향, age: non-linear\n1. subcort + cor_ICV = total_ICV -> 분모 / 분자 : feature들 -> ratio 이용\n   -> age related된 effect 보정 가능 예상\n      thickness 는 잘 보정 x\n      missing data 많은 col 찾기\n   => 보정하고 다시 학습 진행\n2. 10대가 많으면 , 나눠서 실험\n3. 23개 feature -> dictionary.csv 파일 feature들로 새로 생성\n   개수 적으면 RF에서 강력 -> light model이 더 잘나올듯\n   gcn & 다른 모델 학습 진행\n'

In [6]:
# Now we import the cross-validators we want to use, depending on the specific analysis we want to perform. These are based on scikit learn's CV classes (http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)

# Since I am performing gridsearches to optimize my models, I have to use a nested cross-validation loop (inner_cv).
# If you are not doing any nested cross-validation you can ignore the inner_cv's for now and only have to use the outer_cv lines.
random_seed = 0 # By setting this on zero we ensure we have the exact same splits!

# nested(중첩) cross-validation하기 위해 outer_cv, inner_cv 두 개 사용
if cv_type == 0:
    # '1. Outer CV: Site-stratified fixed fold sizes, Inner CV: Site-stratified fixed fold sizes'
    #outer_cv = StratifiedKFoldByGroups(n_splits=10, random_state=random_seed, shuffle=True)
    #inner_cv = StratifiedKFoldByGroups(n_splits=5, random_state=random_seed, shuffle=True)
    outer_cv = StratifiedKFold(n_splits=10, random_state=random_seed, shuffle=True)
    inner_cv = StratifiedKFold(n_splits=5, random_state=random_seed, shuffle=True)
elif cv_type == 1:
    # '2. Outer CV: Leave One Group Out, Inner CV: Group K-Fold'
    outer_cv = LeaveOneGroupOut() # Note that here we don't have to use a random seed, as these splits will always be the same
    inner_cv = GroupKFold(n_splits=5)
elif cv_type == 2:
    # '3. Outer CV: Site-stratified mixed fold sizes, Inner CV: Site-stratified fixed fold sizes'
    outer_cv = StratifiedKFoldMixedSizes(random_state=random_seed)
    inner_cv = StratifiedKFoldByGroups(n_splits=5, random_state=random_seed, shuffle=True)

testAccArr = []
avgValAcc = []
sc = StandardScaler()

for outer_id, (train_id, test_id) in enumerate(outer_cv.split(X, y, cov)):

        print("Outer ID: {}".format(outer_id + 1))
        X_train_val, X_test = X[train_id], X[test_id]
        y_train_val, y_test = y[train_id], y[test_id]
        groups_train_val, groups_test = cov[train_id], cov[test_id]

        if args.model == 'GCN' or args.model == 'DSGC' or args.model == 'ChebyNet':

            X_graph_feat = np.delete(X_train_val,del_idx,1)
            datAdj = computeLaplacian(X_graph_feat,args,dist)  #adjacency Matrix
            valAcc = []
            
            
            for inner_id, (train_id, val_id) in enumerate(inner_cv.split(X_train_val, y_train_val, groups_train_val)):
                print("Inner ID: {}".format(inner_id + 1))
                X_train, X_val = X_train_val[train_id], X_train_val[val_id]
                y_train, y_val = y_train_val[train_id], y_train_val[val_id]
                groups_train, groups_val = groups_train_val[train_id], groups_train_val[val_id]

                X_train = sc.fit_transform(X_train)
                X_val = sc.transform(X_val)
                
                #print(len(X_train), len(X_train[0]))
                
                #initialize weights & bias
                # set a model
                model = eval(args.model).Model(args, datAdj,del_idx)
                model = model.to(device)
                
                # set a loss function & an optimizer
                # optimizer : sgd, adam만 가능
                criterion = nn.CrossEntropyLoss()
                optimizer = Optim(model.parameters(), 'sgd', lr=args.lr, weight_decay=1e-4)

                #GCN
                for epoch in range(args.epochs):
                    train(model,epoch,X_train,y_train)
                    val_acc = test(model, X_val,y_val)
                #valAcc.append(val_acc)
                avgValAcc.append(val_acc)
                #print('Avg validation accuracy:{}({})'.format(np.mean(valAcc),np.std(valAcc)))
                # if test_acc > best_acc:
                #     # best_epoch = epoch
                #     torch.save(model.state_dict(), 'best_'+ str(args.lr)+'.pt')
                #     best = test_acc
                if (epoch == int(args.epochs*0.5) or epoch == int(args.epochs*0.75)):
                    optimizer.updateLearningRate()

                # print('Loading {}th epoch'.format(best_epoch))
                # model.load_state_dict(torch.load('best'+args.dataset+'.pt'))
                # test_acc =
            X_test = sc.transform(X_test)
            testAccArr.append(test(model, X_test,y_test))
        
        else:
            """
            #-----------------------feature selection------------------------
            rfc = RandomForestClassifier()
            rfecv = RFECV(estimator=rfc, step=1, scoring='accuracy')
            rfecv.fit(X_train_val, y_train_val)

            X_feat=[]

            #print(len(X_train_val))      1580
            #print(len(X_train_val[0]))   158
            #print(rfecv.n_features_)      56
            #print(len(rfecv.support_))   158

            for j in range(len(X_train_val)):
                for i in range(len(rfecv.support_)):
                    X_feat_sel=[]
                    if(rfecv.support_[i]==True):
                        X_feat_sel.append(X_train_val[j][i])
                X_feat.append(X_feat_sel)

            #print(X_feat)

            print("feature selection 완료\n")
            
            #-----------------------------------------------------------------
            """
            
            if args.model == "RF":
                params = {'randomforest__min_samples_leaf': np.arange(1, 51, 5),
                          'randomforest__n_estimators': np.arange(10, 100, 10)}

                pipe = Pipeline([
                    ('featureExtract', VarianceThreshold()),
                    ('scaling', StandardScaler()),
                    ('randomforest', RandomForestClassifier(random_state=0))
                ])
            elif args.model == 'SVM':
                params = {'svm__alpha': np.logspace(-4, 7, 12)}

                pipe = Pipeline([
                    ('featureExtract', VarianceThreshold()),
                    ('scaling', StandardScaler()),
                    ("svm", SGDClassifier(max_iter=1000, tol=1e-5, random_state=0))
                ])
            elif args.model == 'LR':
                params = {'lr__C': np.logspace(-3, 8, 12)}

                pipe = Pipeline([
                    ('featureExtract', VarianceThreshold()),
                    ('scaling', StandardScaler()),
                    ('lr', linear_model.LogisticRegression(random_state=0))
                ])
            elif args.model == 'MLP':
                params = {'mlp__hidden_layer_sizes': [[64,],[128,],[64,32],[128,64]]}

                pipe = Pipeline([
                    ('featureExtract', VarianceThreshold()),
                    ('scaling', StandardScaler()),
                    ('mlp', MLPClassifier(random_state=0,early_stopping=True))
                ])

            clf = GridSearchCV(estimator=pipe, param_grid=params, cv=inner_cv, scoring='accuracy', n_jobs=-1)
            clf.fit(X_train_val, y_train_val,groups_train_val)
            print(clf.best_params_)

            fs = clf.best_estimator_.named_steps['featureExtract']
            y_pred = clf.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            testAccArr.append(acc)

Outer ID: 1
==> Computing Laplacian..
Inner ID: 1
[  0/  60]  Training Time: 0.023386 Loss: 0.462 | Acc: 52.174% (84/161) Testing Time: 0.002937 Acc: 48.780% (20/41)
[  1/  60]  Training Time: 0.018594 Loss: 0.449 | Acc: 55.901% (90/161) Testing Time: 0.002963 Acc: 60.976% (25/41)
[  2/  60]  Training Time: 0.016172 Loss: 0.437 | Acc: 63.354% (102/161) Testing Time: 0.002855 Acc: 63.415% (26/41)
[  3/  60]  Training Time: 0.022785 Loss: 0.426 | Acc: 70.807% (114/161) Testing Time: 0.003388 Acc: 63.415% (26/41)
[  4/  60]  Training Time: 0.017047 Loss: 0.430 | Acc: 69.565% (112/161) Testing Time: 0.002876 Acc: 63.415% (26/41)
[  5/  60]  Training Time: 0.016567 Loss: 0.427 | Acc: 67.702% (109/161) Testing Time: 0.002719 Acc: 63.415% (26/41)
[  6/  60]  Training Time: 0.016572 Loss: 0.404 | Acc: 68.323% (110/161) Testing Time: 0.002920 Acc: 58.537% (24/41)
[  7/  60]  Training Time: 0.016425 Loss: 0.394 | Acc: 69.565% (112/161) Testing Time: 0.002779 Acc: 58.537% (24/41)
[  8/  60]  Trai

In [7]:
#if args.model == 'GCN' or args.model == 'DPIEnn' or args.model == 'DSGC' or args.model == 'ChebyNet':
print('Avg valication Accuracy:{}({})'.format(np.mean(avgValAcc),np.std(avgValAcc)))
print('Avg Test Accuracy:{}({})'.format(np.mean(testAccArr),np.std(testAccArr)))

Avg valication Accuracy:63.019512195121955(6.440782280368224)
Avg Test Accuracy:65.88932806324111(10.611820136836542)


In [8]:
fs_labels

array(['subcort_ICV', 'cort_ICV', 'LLatVent', 'Laccumb', 'Lamyg', 'Lcaud',
       'Lhippo', 'Lpal', 'Lput', 'Lthal', 'RLatVent', 'Raccumb', 'Ramyg',
       'Rcaud', 'Rhippo', 'Rpal', 'Rput', 'Rthal', 'LSurfArea',
       'L_bankssts_surfavg', 'L_caudalanteriorcingulate_surfavg',
       'L_caudalmiddlefrontal_surfavg', 'L_cuneus_surfavg',
       'L_entorhinal_surfavg', 'L_frontalpole_surfavg',
       'L_fusiform_surfavg', 'L_inferiorparietal_surfavg',
       'L_inferiortemporal_surfavg', 'L_insula_surfavg',
       'L_isthmuscingulate_surfavg', 'L_lateraloccipital_surfavg',
       'L_lateralorbitofrontal_surfavg', 'L_lingual_surfavg',
       'L_medialorbitofrontal_surfavg', 'L_middletemporal_surfavg',
       'L_paracentral_surfavg', 'L_parahippocampal_surfavg',
       'L_parsopercularis_surfavg', 'L_parsorbitalis_surfavg',
       'L_parstriangularis_surfavg', 'L_pericalcarine_surfavg',
       'L_postcentral_surfavg', 'L_posteriorcingulate_surfavg',
       'L_precentral_surfavg', 'L_precun